In [128]:
import geopandas as gpd
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
import urllib.request

In [129]:
with urllib.request.urlopen("https://www.trusselltrust.org/get-help/find-a-foodbank/foodbank-search/?foodbank_s=all&callback=?") as url:
    data = json.loads(url.read().decode()[2:-2])

In [143]:
fbanklist=[]
for dictionary in data:
    if dictionary['foodbank_centre'] != False:
        for fbank in dictionary['foodbank_centre']:
            try:
                fbank['website'] = dictionary['foodbank_information']['website']
                fbanklist.append(fbank)
            except KeyError:
                continue
    else:
        continue
        
foodbanks = pd.DataFrame(fbanklist)

In [144]:
foodbanks.head()

,foodbank_name,foodbank_telephone_number,opening_time,centre_address,post_code,centre_geolocation,website,local_transport_links
0,Bon Accord Free Church,01224651000,"[{'day': 'wednesday', 'foodbank_status': 'open...","Rosemount Viaduct,<br />\r\nAberdeen<br />\r\n",AB25 1NS,"{'address': 'Rosemount Viaduct Aberdeen, AB25 ...",http://aberdeennorth.foodbank.org.uk/,NaN
1,King's Community Church,01224651000,"[{'day': 'monday', 'foodbank_status': 'open', ...","10 Urquhart Road,<br />\r\nAberdeen,<br />\r\n",AB24 5LL,"{'address': '10 Urquhart Road, Aberdeen AB24 5...",http://aberdeennorth.foodbank.org.uk/,NaN
2,The Mission,01224651000,"[{'day': 'thursday', 'foodbank_status': 'open'...","St Machar Dr, <br />\r\nAberdeen<br />\r\n",AB24 3RX,"{'address': 'St Machar Dr, Aberdeen, AB24 3R...",http://aberdeennorth.foodbank.org.uk/,NaN
3,Tillydrone Campus Foodstore,01224 489546,"[{'day': 'monday', 'foodbank_status': 'open', ...",Tillydrone Community Campus<br />\r\nHayton Ro...,AB24 2UY,"{'address': '40 Hayton Rd, Aberdeen AB24 2TR, ...",http://aberdeennorth.foodbank.org.uk/,NaN
4,Fraserburgh Foodbank Centre *Deliveries Only*,NaN,"[{'day': 'monday', 'foodbank_status': 'closed'...",NaN,AB43 9HP,"{'address': '58 High St, Fraserburgh AB43 9HP,...",http://aberdeenshirenorth.foodbank.org.uk/,NaN


In [25]:
# Regional stats for Trussell Trust food banks
reg_stats = pd.read_html('https://www.trusselltrust.org/news-and-blog/latest-stats/end-year-stats')
reg_stats[0]

,REGION,ADULTS,CHILDREN,TOTAL
0,Scotland,141195,69410,210605
1,Wales,72580,40793,113373
2,Northern Ireland,21592,15191,36783
3,North East,55713,32995,88708
4,North West,139703,83019,222722
5,Yorkshire & Humberside,57252,32589,89841
6,West Midlands,91212,51022,142234
7,East Midlands,47331,28328,75659
8,East,96037,60044,156081
9,London,106737,59775,166512
